# Cleaning up a CSV file with code

We have a CSV file that needs some attention before we analyze it. We also want to weed out some records unrelated to expenditures that we don't need.

Import two libraries for this job: `csv` and `re`.

In [1]:
import csv
import re

We need to get a CSV reader object spun up so that we can do something with the CSV file.

In [2]:
orig_file_name = 'rahm_spending.csv'
csv_file = open(orig_file_name, 'rb')
infile = csv.reader(csv_file)

We've generally been interacting with CSV files by running straight through every row; we can also step through them line by line with `next()`. Let's take one step down into the file to grab headers; it also means when we run through the rest of the file, we'll start with the first data row.

In [3]:
headers = infile.next()

We're going to go ahead and write a cleaning function to run on the rows of this file. Let's assess what we're going to need to fix. We'll only mess with columns that have problems.

- **LASTONLYNAME** needs to be uppercase
- FIRSTNAME
- EXPENDEDDATE
- **AMOUNT** suffers from whitespace and dollar signs
- ADDRESS1
- ADDRESS2
- **CITY** contains some problematic spellings of 'Chicago' and non-breaking spaces for display (`&NBSP;`)
- STATE
- **ZIP** has leading zeros removed
- EXPTYPE
- **PURPOSE** has additional detail that can be split into another column and synonyms that can really be standardized for fewer categories during analysis
- BENEFICIARY

In [4]:
def cleaner(row):

    lastonlyname = row[0].upper()

    amount = float(row[3].replace('$', '').strip())

    if row[6] in ['CHGO', 'CHCAGO']:
        city = 'CHICAGO'
    else:
        city = row[6].replace('&NBSP;', ' ')
        
    if len(row[8]) == 4:
        zip = '0{}'.format(row[8])
    else:
        zip = row[8]

    p_split = re.split('-|/', row[10])
    if len(p_split) > 1:
        main_purpose = p_split[0].strip()
        purpose_extra = p_split[1].strip()
    else:
        main_purpose = row[10]
        purpose_extra = ''
    problem_words = ['FEE', 'FEES', 'COST', 'COSTS', 'EXPENSE']
    purpose_words = main_purpose.split()
    for word in purpose_words:
        if word in problem_words:
            loc = purpose_words.index(word)
            purpose_words.pop(loc)
            purpose_words.insert(loc, 'EXPENSES')
    main_purpose = ' '.join(purpose_words)

    cleaned_row = [lastonlyname, row[1], row[2], amount, row[4], row[5], city, row[7], zip, row[9], main_purpose, purpose_extra, row[11]]
    return cleaned_row

Our file is loaded and ready to go. We have a cleaning function. Let's fix the headers to match our file changes before we loose the function on the file.

In [5]:
headers.insert(headers.index('PURPOSE') + 1, 'DETAIL')

Now we can go ahead and modify our rows before the hit a new file. We'll also check to see if they are expenditures; if so, we'll write them to the file.

In [6]:
clean_file_name = 'rahm_spending_clean.csv'
with open(clean_file_name, 'wb') as outfile:
    output = csv.writer(outfile)
    output.writerow(headers)
    for row in infile:
        # Here's where we can weed out non-expenditures from hitting our clean file.
        if row[9] == 'EXPENDITURE':
            output.writerow(cleaner(row))

print('All done!')
outfile.close()

All done!
